# Programming Practice Problems in Python, Part 2

**Author:** Michelle Franc Ragsac (mragsac@eng.ucsd.edu)

This notebook contains some additional introductory exercises to basic concepts of programming, specifically for the Python programming language, but with a bioinformatics twist!

> * Taken from the TA materials from the CMM262: Quantitative Methods in Genetics course at UCSD: https://github.com/biom262/cmm262-2021/tree/main/ta-materials/01-15-2021_python-practice
> * Adapted from BISB Bootcamp 2020 Day 4, Module 6: Pair Programming materials: https://github.com/mragsac/BISB-Bootcamp-2020/tree/master/day4/module6_pair-programming
> * Many of these exercises (and more!) can be found through the Rosalind "Bioinformatics Stronghold" resource: http://rosalind.info/problems/list-view/?location=bioinformatics-stronghold

---

### [Counting DNA Nucleotides](http://rosalind.info/problems/dna/)

<div class="alert alert-block alert-success">
    <p><b>Exercise:</b></p>
    <p><b><i>Given</i></b>: A DNA sequence string, <code>s</code>, with a maximum length of 1,000 nucleotides 
    <br><b><i>Return</i></b>: Four integers (separated by spaces) counting the respective number of times that the bases <code>A</code>, <code>T</code>, <code>C</code>, and <code>G</code> occur in <code>s</code></p>
</div>

In [41]:
import pandas as pd
from collections import Counter, defaultdict
import numpy as np
import cudf

str_size = int(1e8)
dna_str = np.random.choice(list("ATCG"), str_size)

In [42]:
%%timeit 
pd.Series(dna_str).value_counts()

7.42 s ± 95.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
Counter(dna_str)

19.4 s ± 438 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
d = defaultdict(int)
for c in dna_str:
    d[c] + 1
d

17.2 s ± 30.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
import cudf

In [46]:
%%timeit
cudf.Series(dna_str).value_counts()

5.41 s ± 61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
s = cudf.Series(dna_str)

In [48]:
%%timeit
s.value_counts()

56.3 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


---

### [Compute the Hamming Distance Between Two Strings](http://rosalind.info/problems/ba1g/)

We say that position $i$ in k-mers $p_1 … p_k$ and $q_1 … q_k$ is a mismatch if $p_i ≠ q_i$. For example, `CGAAT` and `CGGAC` have two mismatches. 

The number of mismatches between strings $p$ and $q$ is called the **Hamming Distance** between these strings and is denoted `HammingDistance(p, q)`.

<div class="alert alert-block alert-success">
    <p><b>Exercise:</b></p>
    <p><b><i>Given</i></b>: Two DNA Strings
    <br><b><i>Return</i></b>: An integer value representing the Hamming Distance of the two DNA strings</p>
</div>

In [49]:
from scipy.spatial.distance import hamming

In [79]:
str_size = int(1e7)
s1 = np.random.choice(list("ATCG"), str_size)
s2 = np.random.choice(list("ATCG"), str_size)

In [80]:
%%time
sum(c1!=c2 for c1,c2 in zip(s1,s2))

CPU times: user 3.29 s, sys: 0 ns, total: 3.29 s
Wall time: 3.29 s


7501049

In [81]:
%%time
(pd.Series(list(s1)) != pd.Series(list(s2))).sum()

CPU times: user 4.83 s, sys: 746 ms, total: 5.58 s
Wall time: 5.54 s


7501049

In [82]:
%%time
(cudf.Series(list(s1)) != cudf.Series(list(s2))).sum()

CPU times: user 3.25 s, sys: 689 ms, total: 3.94 s
Wall time: 3.91 s


7501049

In [86]:
%%time
hamming(s1,s2) * len(s1)

CPU times: user 95.1 ms, sys: 3.98 ms, total: 99 ms
Wall time: 95.9 ms


7501049.0

---

### [Complementing a Strand of DNA](http://rosalind.info/problems/revc/)

In DNA strings, symbols `A` and `T` are complements of each other, as are `C` and `G`. The reverse complement of a DNA string $s$ is the string $s_c$ formed by reversing the symbols of `s`, then taking the complement of each symbol (e.g., the reverse complement of `GTCA` is `TGAC`).

<div class="alert alert-block alert-success">
    <p><b>Exercise:</b></p>
    <p><b><i>Given</i></b>: A DNA string, <code>s</code>, with a maximum length of 1,000 nucleotides
    <br><b><i>Return</i></b>: The reverse complement of <code>s</code></p>
</div>

In [99]:
s1 = 'GTCA'
#str_size = int(1e7)
#s1 = np.random.choice(list("ATCG"), str_size)

In [100]:
%%time
comp_map = {'A': 'T', 'T': 'A', 'C':'G', 'G':'C'}
''.join(pd.Series(list(s1)).map(comp_map).values)

CPU times: user 1.75 ms, sys: 41 µs, total: 1.79 ms
Wall time: 1.81 ms


'CAGT'

In [101]:
%%time
s2 = []
for c in s1:
    s2.append(comp_map.get(c, c))
''.join(s2)

CPU times: user 13.7 ms, sys: 3.96 ms, total: 17.7 ms
Wall time: 17.4 ms


'CAGT'

---

### [Find the Most Frequent "Words" in a String](http://rosalind.info/problems/revc/)

**k-mers** are small subsequences of a string that are of length `k`. We can define the term *Pattern* as our most frequent k-mer in a string if it maximizes `Count(Text,Pattern)` among all k-mers. For example, `ACTAT` is a most frequent 5-mer in `ACAACTATGCATCACTATCGGGAACTATCCT`, and `ATA` is a most frequent 3-mer of `CGATATATCCATAG`.

<div class="alert alert-block alert-success">
    <p><b>Exercise:</b></p>
    <p><b><i>Given</i></b>: A DNA string, <code>Text</code>, and an integer, <code>k</code>
    <br><b><i>Return</i></b>: All most frequent k-mers in <code>Text</code> in any order
</div>

In [125]:
#s1 = 'CGATATATCCATAG'
str_size = int(1e5)
s1 = np.random.choice(list("ATCG"), str_size)
s1 = ''.join(list(s1))

In [126]:
kmers_len = 5
kmers = defaultdict(int)
for i in range(0,len(s1)-kmers_len+1):
    kmer = s1[i:(i+kmers_len)]
    kmers[kmer] += 1

pd.Series(kmers).sort_values(ascending=False)

GTGTC    130
GTACG    130
AGGCT    129
GCACA    127
CACAA    127
        ... 
CGACC     70
GGGGC     69
TGGTA     68
CGGGG     64
TATAA     62
Length: 1024, dtype: int64